# DynamoDB Callback

The notebook showcases how to use the [StreamingDynamoDBCallbackHandler](./slangchain/callbacks/streaming_aws_ddb.py) class. The functionality streams callbacks to a AWS DyanmoDB table

[LangChain](https://github.com/hwchase17/langchain) provides a callback system that allows you to hook into the various stages of your LLM application. This is useful for logging, [monitoring](https://python.langchain.com/en/latest/tracing.html), [streaming](https://python.langchain.com/en/latest/modules/models/llms/examples/streaming_llm.html), and other tasks.

## Prerequisites

- [An AWS account](https://aws.amazon.com/account/sign-up)
- [AWS credentials setup](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html)

## AWS Setup

Create a test DyanmoDB table for this example.

In [5]:
import boto3

table_name = "langchain_callback_datastore"

ddb_resource = boto3.resource("dynamodb")
ddb_resource.create_table(
  TableName = table_name,
  KeySchema=[
    {'AttributeName': 'partition_key', 'KeyType': 'HASH'},  # Partition key
    {'AttributeName': 'sort_key', 'KeyType': 'RANGE'}  # Sort key
  ],
  AttributeDefinitions=[
    {'AttributeName': 'partition_key', 'AttributeType': 'S'},
    {'AttributeName': 'sort_key', 'AttributeType': 'S'}
  ],
  ProvisionedThroughput={'ReadCapacityUnits': 10, 'WriteCapacityUnits': 10}
)


dynamodb.Table(name='langchain_callback_datastore')

## Example

Create an on agent action callback (You might have to wait until the DynamoDB table is created before running the below code)

In [6]:
from uuid import uuid4
from langchain.schema import AgentAction
from slangchain.callbacks.streaming_aws_ddb import StreamingDynamoDBCallbackHandler

# set a partition key value
partition_key_value = str(uuid4())
# set a sort key value
sort_key_prefix_value = str(uuid4())

handler = StreamingDynamoDBCallbackHandler(
    ddb_table_name=table_name,
    partition_key_value=partition_key_value,
    sort_key_prefix_value=sort_key_prefix_value,
)

handler.on_agent_action(AgentAction("tool", "tool_input", "log"), run_id="test")

Validate that the item has been created in the DynamoDB table

In [7]:
from slangchain.aws.dynamodb.base import DDBAppLoader

ddb_loader = DDBAppLoader(
  table_resource=ddb_resource.Table(table_name),
  partition_key_value=partition_key_value,
  sort_key_value=sort_key_prefix_value)

ddb_loader.load()

DDBPayloadList(__root__=[DDBPayload(partition_key='0ccf6dc2-e0df-42c1-8837-766cff45187e', sort_key='941b7b0f-e7d8-4a10-80f2-b906713d087e#8cecb10b-e689-489b-9917-0671b5be4207#1683018356.762323#AGENT_ACTION#000001', record_type='AGENT_ACTION', timestamp=1683018356.762323, sequence_number=1, payload={'message': {'tool_input': 'tool_input', 'log': 'log', 'tool': 'tool'}, 'arguments': {}})])

Cleaning up AWS resources

In [4]:
client = boto3.client("dynamodb")
client.delete_table(
    TableName=table_name
)

{'TableDescription': {'TableName': 'langchain_callback_datastore',
  'TableStatus': 'DELETING',
  'ProvisionedThroughput': {'NumberOfDecreasesToday': 0,
   'ReadCapacityUnits': 10,
   'WriteCapacityUnits': 10},
  'TableSizeBytes': 0,
  'ItemCount': 0,
  'TableArn': 'arn:aws:dynamodb:ap-southeast-2:734145044054:table/langchain_callback_datastore',
  'TableId': '292fa492-1831-4bf4-813e-44835b232894'},
 'ResponseMetadata': {'RequestId': '4DJND95N5ULF7021RLRVDAS24NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Tue, 02 May 2023 09:05:26 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '399',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4DJND95N5ULF7021RLRVDAS24NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2782254370'},
  'RetryAttempts': 0}}